In [7]:
import dotenv

dotenv.load_dotenv()

True

## Assemble RAG Pipeline

In [96]:
from llama_index.response_synthesizers import TreeSummarize
from rag import weaviate_utils
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.query_pipeline import QueryPipeline, InputComponent
from llama_index.llms import OpenAI

weaviate_client = weaviate_utils.get_weaviate_client()
vector_store = weaviate_utils.as_vector_store(weaviate_client, class_name="DocsChunk")
index = VectorStoreIndex.from_vector_store(vector_store)
retriever = index.as_retriever(similarity_top_k=5)

llm = OpenAI(model="gpt-3.5-turbo")
summarizer = TreeSummarize(
    service_context=ServiceContext.from_defaults(llm=llm)
)

# assumes that the database has already been populated
pipeline = QueryPipeline(verbose=True)
pipeline.add_modules({
    "input": InputComponent(),
    "retriever": retriever,
    "summarizer": summarizer,
})

pipeline.add_link("input", "retriever")
pipeline.add_link("input", "summarizer", dest_key="query_str")
pipeline.add_link("retriever", "summarizer", dest_key="nodes")

In [109]:
response = pipeline.run(input="What is ECS?")
response.response

> Running module input with input: 
input: What is ECS?

> Running module retriever with input: 
input: What is ECS?

> Running module summarizer with input: 
query_str: What is ECS?
nodes: [NodeWithScore(node=TextNode(id_='d6c6fefd-5661-4ffe-b2e2-445b25ab1a4f', embedding=[0.012504828, 0.010405679, -0.008479728, -0.016335947, -0.005947587, 0.012504828, -0.013017491, -0.009047815, -0.0069...



'ECS stands for Environmental Control System. It is a vital component of the spacecraft module that is responsible for creating a comfortable and safe environment for both astronauts and delicate electronic components onboard the spacecraft. The ECS oversees various aspects of maintaining a hospitable environment in the unforgiving vacuum of space, including stable life support and protection of electronic equipment.'

## Evaluation

In [98]:
from trulens_eval import Tru

tru = Tru()

In [99]:
from trulens_eval.feedback.provider.openai import OpenAI

openai = OpenAI()

In [100]:
from trulens_eval import Feedback

# response relevance
f_qa_relevance = Feedback(openai.relevance, name="Response Relevance").on_input_output()

✅ In Response Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Response Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [101]:
from trulens_eval import TruLlama, TruBasicApp
from typing import Callable


def tru_query_pipeline_wrapper(query_pipeline: QueryPipeline) -> Callable[[str], str]:
    def run(prompt: str) -> str:
        response = query_pipeline.run(input=prompt)
        return response.response

    return run


tru_recorder = TruBasicApp(
    tru_query_pipeline_wrapper(query_pipeline=pipeline),
    app_id="pipeline v1",
    feedbacks=[f_qa_relevance]
)

In [102]:
with tru_recorder as recording:
    response = tru_recorder.app("How can I turn off the ECS controller?")
    print(response)

> Running module input with input: 
input: How can I turn off the ECS controller?

> Running module retriever with input: 
input: How can I turn off the ECS controller?

> Running module summarizer with input: 
query_str: How can I turn off the ECS controller?
nodes: [NodeWithScore(node=TextNode(id_='4db1ec04-695a-4e87-8484-cec616bedd38', embedding=[0.012828164, 0.0089711435, -0.006985492, -0.02905622, -0.00053569727, 0.024070663, -0.0070604896, -0.02449922, -0.00...

To turn off the ECS controller, you can use the `set` command with the appropriate configuration. However, the specific command to turn off the ECS controller is not provided in the given context information.


In [103]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.44:8501


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [104]:
tru.stop_dashboard()